In [ ]:
import os
import time
import json 
import regex
import random
import pickle 
import re
import sys
import argparse
import numpy as np
import openai
from func_timeout import func_timeout, FunctionTimedOut
import pandas as pd
from tqdm import tqdm, trange
from utils import *
import logging

In [ ]:
# Code for Multi agent debate
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = ""
openai.api_key = ""

def gpt4(text):
    
    try:
        response = openai.ChatCompletion.create(
          engine="gpt-4-32k",
          messages= [
        {"role": "user", "content": text}
    ],
          temperature=0.7,
          max_tokens=6000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as exc:
        print(exc)
        time.sleep(30)
        return gpt4(text)
    


def get_res_from_other_agents(gpt4_res, get_prompt_for_agent):
    
    final_prompt = "These are responses from other agents."
    agent_c = 1
    for i in range(len(gpt4_res)):
        if i != get_prompt_for_agent:
            final_prompt += f"\n Agent{agent_c} :" + gpt4_res[i] 
            agent_c += 1
    
    final_prompt += "\n Based on the opinion of other agents , give an updated response"
    return final_prompt


def output(init_prompt, no_of_agents):
    
    gpt4_res = [gpt4(init_prompt) for i in range(no_of_agents)]
    
    folder_name = f"{no_of_agents}_Model_multi_agent_output"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    gpt4_1_file = os.path.join(folder_name, "gpt4_1.txt")
    
    ## Initial prompt
    with open(gpt4_1_file, 'w') as file:
        file.write("Initial prompt >>>>>>>> "+ init_prompt)
        file.write('*'*500)
        file.write("\n\n")
    
    for itr in trange(1, 6):
        gpt4_input = [get_res_from_other_agents(gpt4_res, i) for i in range(no_of_agents)]

        gpt4_res = [gpt4(inp) for inp in gpt4_input]
        
        with open(gpt4_1_file, 'a') as file:
            file.write("After iteration >>>>>>>>>>>>>>>> "+ str(itr))
            file.write("\n")
            file.write(gpt4_res[0])
            file.write("\n")
            file.write('~'*500)
            file.write("\n")

    with open(gpt4_1_file, 'r') as file:
        return_text = file.read()
    
    return gpt4_res[0], return_text

In [ ]:
class Autoplan:
    def __init__(self, lm, method, backend_args, quota_args, **kwargs):
        self.lm = lm
        self.method = method  # step
        self.backend = backend_args['name']  # openai
        
        if self.backend == 'openai':
            openai.api_type = "azure"#quota_args["api_type"]
            openai.api_base = quota_args["api_base"]
            openai.api_version = quota_args["api_version"]
            openai.api_key = quota_args["api_key"]
    
        self.top_p = backend_args['top_p']
        self.temp = backend_args['temp']
        self.max_token = backend_args['max_token']
        self.presence_penalty = backend_args['presence_penalty']        
        
        self.max_iter_per_instance = quota_args['max_iter_per_instance']

        
        self.history = []
        self.strategy = None

        # openai api
        self.n_prompt_token = 0
        self.n_sample_token = 0
        self.messages = []
        
    def task_desription(self, sub):
        if(sub=="chem"):
            return '''Solve a Chemistry question answering task'''
        elif(sub=="math"):
            return '''Solve a Mathematics question answering task'''
        else:
             return '''Solve a Physics question answering task'''         
       
    def price(self):
        price = PRICE[self.lm]
        return (self.n_prompt_token * price['prompt'] + self.n_sample_token * price['sample']) / 1000
    
    
    def call_openai_api(self, messages, stop, lm=None, top_p=None):
        n_try = 10
        while n_try > 0:
            try:
                #time.sleep(1)
                response =  openai.ChatCompletion.create(
                       engine="gpt-4-turbo",  # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
                        messages=messages,
                        temperature=self.temp,
                        max_tokens= self.max_token,
                        presence_penalty=self.presence_penalty,
                        stop=stop,
                    )
                
                break
            except FunctionTimedOut:
                print('[LOG] OpenAI API call timeout')
                n_try -= 1
                if n_try == 0:
                    raise Exception('Failed 10 retries.')
                continue
            except Exception as e:
                print('[LOG]', e)
                time.sleep(15)
                n_try -= 1
                if n_try == 0:
                    raise Exception('Failed 10 retries.')
                continue
        return response
        
    def call_lm(self, prompt, add_response=True, stop=None, lm=None, top_p=None):
        
        self.messages.append({'role': 'user', 'content': prompt})
        response = self.call_openai_api(self.messages, stop, lm=lm, top_p=top_p)
        if add_response: self.messages.append(response['choices'][0]['message'])
        self.n_prompt_token += response['usage']['prompt_tokens']
        self.n_sample_token += response['usage']['completion_tokens']
        return response['choices'][0]['message']['content']
    
    def run(self, data, strategy=None, is_test=False, verbose=False, return_history=False):
        questions = []
        answers = []
        predictions = []
        summaries = []
        flawed_actions = []
        flawed_plans = []
        history = ''

        
        for q_idx in range(len(data)):
            self.messages = []
            sub = data[q_idx]['subject']
            init_msg = self.task_desription(sub)
            if 'direct' not in self.method and strategy is not None:
                init_msg += "\n\nTask Plan:\n{}\n".format(strategy)
            init_msg += '\n'

            question = data[q_idx]['question']
            answer = data[q_idx]['gold']
            questions.append(question)
            answers.append(answer)
            
            input_msg = init_msg + '\nQuestion: ' + question + '\n'
            solution, all_res = output(input_msg, 4)
            
            logger.info(pretty_print('Human', input_msg, verbose))
            logger.info(solution)
            
            if not is_test:
                summary_msg = ' The ground truth solution is "{}". Summarize the interaction history concisely.'.format(answer)
                logger.info(pretty_print('Human', summary_msg, verbose))
                summary, all_summary = output(summary_msg, 4)
                logger.info(pretty_print('Machine', summary, verbose))
                summaries.append(summary)
                
                if strategy is not None:
                    failed_action_msg = 'Identify all flawed parts of the plan (not flawed action). Only the flawed part.'
                    logger.info(pretty_print('Human', failed_action_msg, verbose))
                    failed_action, all_failed = output(failed_action_msg, 4)
                    logger.info(pretty_print('Machine', failed_action, verbose))
                    flawed_actions.append(failed_action)
                    suggest_rev_msg = 'Suggest revision to the current flawed part of the plan. Only the flawed part.'
                    logger.info(pretty_print('Human', suggest_rev_msg, verbose))
                    suggest_rev, all_revs = output(suggest_rev_msg, 4)
                    logger.info(pretty_print('Machine', suggest_rev, verbose))
                    flawed_plans.append(suggest_rev)

        to_return = None

        if is_test:
            to_return = predictions
        else:
            self.messages = []
            final_msg = 'Task Description:\n' + self.task_desription(sub) + '\n\n'
            final_msg += 'Current Task Plan:\n{}\n\n'.format(strategy)
            final_msg += '=' * 10 + 'Task Experiences Begin' + '=' * 10 + '\n\n'

            for q_idx in range(len(data)):
                question = data[q_idx]['question']
                final_msg += 'Job {}: Answering the following question. {}\nSummary of Job {}:\n{}\n'.format(q_idx, question, q_idx, summaries[q_idx])
                if strategy is not None:
                    final_msg += 'Flaws of Plan in Job {}:\n{}\n'.format(q_idx, flawed_actions[q_idx])
                    final_msg += 'Suggested Revision of Plan from Job {}:\n{}\n'.format(q_idx, flawed_plans[q_idx])

            final_msg += '=' * 10 + 'Task Experiences End' + '=' * 10 + '\n\n'
            final_msg += 'Based on the above {} experience of the task, rewrite the current task plan. The plan should be specific to this question and instead of providing the solution give a specific approach.  \nNew Task Plan:'.format(len(data))
            logger.info(pretty_print('Human', final_msg, verbose))

            new_strategy, new_strats = output(final_msg, 4)
            logger.info(pretty_print('Machine', new_strategy, verbose))
            to_return = new_strategy

        self.history.append(history)

        if return_history:
            to_return = to_return, history
            return to_return
        return to_return, history

In [ ]:
backend_args = {
        'name': "openai",
        'top_p': 1,
        'temp': 0,
        'max_token': 3000,
        'presence_penalty': 1.5,
    }
quota_args = {
        'sleep_minutes': 1,
        'max_iter_per_instance': 4,
        "engine": open_ai_ids.iloc[args.i]['Deployment'],
        "api_base": open_ai_ids.iloc[args.i]['Base'],
        "api_version": open_ai_ids.iloc[args.i]['Version'],
        "api_key":  open_ai_ids.iloc[args.i]['Key']
}


In [ ]:
with open("../dataset/dataset.json", "r") as f:
    data = json.load(f)
    
data_train = data[:360]

for batch in trange(start, end):
    
    autoplan = Autoplan("gpt-4-turbo", 'step', backend_args, quota_args)
    
    strategy = "Let's first understand the problem and devise a plan to solve the problem. Then, let's carry out the plan and solve the problem step by step."          # Plan and solve prompting..
    
    new_strategy, history = autoplan.run([data_train[batch]],
                                    strategy=strategy,
                                    is_test=False,
                                    verbose=False,
                                    return_history=True)
    
    data_train[batch]['strategy'] = new_strategy
    
    with open(f"./strategies/AutoMAD/strategy_{start}_{end}.json", 'w') as f:
            json.dump(data_train[start:end], f)
        